In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#dependency
!pip3 install -q -U bitsandbytes==0.42.0
!pip3 install -q -U peft==0.8.2
!pip3 install -q -U trl==0.7.10
!pip3 install -q -U accelerate==0.27.1
!pip3 install -q -U datasets==2.17.0
!pip3 install -q -U transformers==4.38.0

In [ ]:
import pandas as pd
import numpy as np
import torch
import transformers
import bitsandbytes as bnb
import os
from transformers import PreTrainedTokenizerFast, AdamW, AutoModelForCausalLM, BitsAndBytesConfig
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model
from tqdm import tqdm

from peft import PeftConfig, PeftModel
checkpoint_path = "/content/drive/MyDrive/checkpoint-1832"
peft_config = PeftConfig.from_pretrained(checkpoint_path)
base_model_path = peft_config.base_model_name_or_path
print(base_model_path)

google/gemma-2b


In [ ]:
import os
from google.colab import userdata
os.environ["HF_TOKEN"] = userdata.get('HF_TOKEN_READ')

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

model_id = 'google/gemma-2b'
model = AutoModelForCausalLM.from_pretrained(model_id,
                                             quantization_config=bnb_config,
                                             device_map={"":0},
                                             token=os.environ['HF_TOKEN'],
                                             )
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)
peft_model = PeftModel.from_pretrained(model, checkpoint_path)
tokenizer = AutoTokenizer.from_pretrained(model_id,token=os.environ['HF_TOKEN'])

config.json:   0%|          | 0.00/627 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/13.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.11k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/555 [00:00<?, ?B/s]

In [ ]:
# CUDA 사용 가능 여부 확인
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
peft_model.to(device)

Using device: cuda


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): GemmaForCausalLM(
      (model): GemmaModel(
        (embed_tokens): Embedding(256000, 2048, padding_idx=0)
        (layers): ModuleList(
          (0-17): 18 x GemmaDecoderLayer(
            (self_attn): GemmaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=2048, out_features=2048, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2048, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=2048, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_fea

# Inference

In [ ]:
test_data_path = "/content/drive/MyDrive/test.csv"
test_data = pd.read_csv(test_data_path)
test_data['질문'] = test_data['질문'].str.strip('"')

In [ ]:
def split_question(row):
    # 원본 문장에서 마침표와 물음표의 위치를 찾습니다.
    dot_index = row['질문'].find('. ')
    question_index = row['질문'].find('? ')

    # 문장 부호의 위치에 따라 분리 로직을 결정합니다.
    if dot_index != -1 and (question_index == -1 or dot_index < question_index):
        # 마침표를 기준으로 분리하되, 마침표 위치를 고려합니다.
        parts = [row['질문'][:dot_index+1].strip(), row['질문'][dot_index+2:].strip()]
    elif question_index != -1:
        # 물음표를 기준으로 분리하되, 물음표 위치를 고려합니다.
        parts = [row['질문'][:question_index+1].strip(), row['질문'][question_index+2:].strip()]
    else:
        # 문장 부호가 없는 경우, 원본 문장을 그대로 사용합니다.
        parts = [row['질문'], None]

    # 분리된 문장을 새로운 칼럼에 할당합니다.
    return pd.Series({'문장1': parts[0], '문장2': parts[1]})

# 적용
new_columns = test_data.apply(split_question, axis=1)
df = pd.concat([test_data, new_columns], axis=1)

df_question_1 = df[df['문장2'].isnull()] # 질문 1개
df_question_2 = df[df['문장2'].notnull()] # 질문 2개

In [ ]:
#df_question_2의 '문장2' 서두의 '또한' & '그리고' 삭제
df_question_2['문장2'] = df_question_2['문장2'].apply(lambda x: x[4:] if x.startswith('그리고') else x)
df_question_2['문장2'] = df_question_2['문장2'].apply(lambda x: x[4:] if x.startswith('또한,') else x)

<ipython-input-9-72e13e1ae2fa>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_question_2['문장2'] = df_question_2['문장2'].apply(lambda x: x[4:] if x.startswith('그리고') else x)
<ipython-input-9-72e13e1ae2fa>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_question_2['문장2'] = df_question_2['문장2'].apply(lambda x: x[4:] if x.startswith('또한,') else x)


## df_question_1

In [ ]:
# test.csv의 '질문'에 대한 '답변'을 저장할 리스트
answer_1 = []

# '질문' 컬럼의 각 질문에 대해 답변 생성
for test_question in tqdm(df_question_1['질문']):
    input_ids = tokenizer.encode(test_question + tokenizer.eos_token, return_tensors='pt')
    # 답변 생성
    output = peft_model.generate(
        input_ids=input_ids.to(device),
        max_length=200,
        repetition_penalty=1.2
    )
    full_text = tokenizer.decode(output[0], skip_special_tokens=False)
    answer_start = full_text.find(tokenizer.eos_token) + len(tokenizer.eos_token)
    answer_only = full_text[answer_start:].strip()
    answer_only = answer_only.replace('\n', ' ')
    answer_1.append(answer_only)

100%|██████████| 67/67 [15:27<00:00, 13.84s/it]


In [ ]:
df_question_1['대답'] = answer_1
df_question_1.to_csv('/content/drive/MyDrive/0307-gemma/answer1.csv')

<ipython-input-13-7e4423751f6c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_question_1['대답'] = answer_1


## df_question_2

In [ ]:
# test.csv의 '질문'에 대한 '답변'을 저장할 리스트
answer_2_1 = []

# '질문' 컬럼의 각 질문에 대해 답변 생성
for test_question in tqdm(df_question_2['문장1']):
    input_ids = tokenizer.encode(test_question + tokenizer.eos_token, return_tensors='pt')
    # 답변 생성
    output = peft_model.generate(
        input_ids=input_ids.to(device),
        max_length=125,
        repetition_penalty=1.2
    )
    full_text = tokenizer.decode(output[0], skip_special_tokens=False)
    answer_start = full_text.find(tokenizer.eos_token) + len(tokenizer.eos_token)
    answer_only = full_text[answer_start:].strip()
    answer_only = answer_only.replace('\n', ' ')
    answer_2_1.append(answer_only)

df_question_2['대답_1'] = answer_2_1
df_question_2.to_csv('/content/drive/MyDrive/0307-gemma/answer_2_1.csv')

100%|██████████| 63/63 [07:45<00:00,  7.38s/it]
<ipython-input-14-c9fa44f7eecf>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_question_2['대답_1'] = answer_2_1


In [ ]:
# test.csv의 '질문'에 대한 '답변'을 저장할 리스트
answer_2_2 = []

# '질문' 컬럼의 각 질문에 대해 답변 생성
for test_question in tqdm(df_question_2['문장2']):
    input_ids = tokenizer.encode(test_question + tokenizer.eos_token, return_tensors='pt')
    # 답변 생성
    output = peft_model.generate(
        input_ids=input_ids.to(device),
        max_length=125,
        repetition_penalty=1.3
    )
    full_text = tokenizer.decode(output[0], skip_special_tokens=False)
    answer_start = full_text.find(tokenizer.eos_token) + len(tokenizer.eos_token)
    answer_only = full_text[answer_start:].strip()
    answer_only = answer_only.replace('\n', ' ')
    answer_2_2.append(answer_only)

df_question_2['대답_2'] = answer_2_2
df_question_2.to_csv('/content/drive/MyDrive/0307-gemma/answer_2_2.csv')

100%|██████████| 63/63 [08:13<00:00,  7.84s/it]
<ipython-input-16-0a92756e3b80>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_question_2['대답_2'] = answer_2_2


In [ ]:
q1 = pd.read_csv('/content/drive/MyDrive/0307-gemma/answer1.csv')
q1 = q1[['id', '대답']]
q1.head(2)

,id,대답
0,TEST_001,"도배지에 녹은 자국이 발생하는 것은 녹 오염으로, 주로 습기와 온도의 변화에 기인합..."
1,TEST_003,"철골 구조는 뼈대 형태의 건축물로, 열 전달을 차단하여 단열 효율이 떨어집니다. 따..."


In [ ]:
q2_1 = pd.read_csv('/content/drive/MyDrive/0307-gemma/answer_2_1.csv')
q2_1 = q2_1[['id', '대답_1']]
q2_1.head(2)

,id,대답_1
0,TEST_000,"방청 페인트의 종류로는 광명단 페인트, 방청산화철 페인트, 알미늄 페인트, 역청질 ..."
1,TEST_002,"큐블럭은 일반 벽돌에 비해 높은 비용과 균열이 생기는 염려점이 있습니다. 또한, 습..."


In [ ]:
q2_2.iloc[21,:]

id      TEST_040
대답_2         NaN
Name: 21, dtype: object

In [ ]:
q2_2 = pd.read_csv('/content/drive/MyDrive/0307-gemma/answer_2_2.csv')
q2_2 = q2_2[['id', '대답_2']]
q2_2.head(2)

,id,대답_2
0,TEST_000,"원목사이딩의 단점은 주로 가격대가 높고 관리가 힘들다는 점으로 언급됩니다. 또한, ..."
1,TEST_002,압출법 단열판의 장점으로는 가격이 저렴하고 시공이 간편하며 내부식성과 내화성이 좋고...


In [ ]:
q2 = q2_1.copy()
q2['대답_2'] = q2_2['대답_2']
q2['대답'] = q2['대답_1'] + q2['대답_2']
q2.head(2)

,id,대답_1,대답_2,대답
0,TEST_000,"방청 페인트의 종류로는 광명단 페인트, 방청산화철 페인트, 알미늄 페인트, 역청질 ...","원목사이딩의 단점은 주로 가격대가 높고 관리가 힘들다는 점으로 언급됩니다. 또한, ...","방청 페인트의 종류로는 광명단 페인트, 방청산화철 페인트, 알미늄 페인트, 역청질 ..."
1,TEST_002,"큐블럭은 일반 벽돌에 비해 높은 비용과 균열이 생기는 염려점이 있습니다. 또한, 습...",압출법 단열판의 장점으로는 가격이 저렴하고 시공이 간편하며 내부식성과 내화성이 좋고...,"큐블럭은 일반 벽돌에 비해 높은 비용과 균열이 생기는 염려점이 있습니다. 또한, 습..."


In [ ]:
q2.iloc[21,1]

'원목마루는 주로 첫 번째 바닥재료로 사용되며, 나무 뿌리(또는 줄기)로 만들어진 바닥재입니다. 일반적으로 평당 가격은 10~20만원으로 알려져 있습니다. 이러한 마루는 따뜻한 느낌을 주고 공간에 아늑함을 더해줍니다. 또한, 나무의 특성상 보행감이 좋고 충격을 흡수하는'

In [ ]:
q2.iloc[21,3]

'원목마루는 주로 첫 번째 바닥재료로 사용되며, 나무 뿌리(또는 줄기)로 만들어진 바닥재입니다. 일반적으로 평당 가격은 10~20만원으로 알려져 있습니다. 이러한 마루는 따뜻한 느낌을 주고 공간에 아늑함을 더해줍니다. 또한, 나무의 특성상 보행감이 좋고 충격을 흡수하는'

In [ ]:
q2.iloc[21,3] ="원목마루는 주로 첫 번째 바닥재료로 사용되며, 나무 뿌리(또는 줄기)로 만들어진 바닥재입니다. 일반적으로 평당 가격은 10~20만원으로 알려져 있습니다. 이러한 마루는 따뜻한 느낌을 주고 공간에 아늑함을 더해줍니다. 또한, 나무의 특성상 보행감이 좋고 충격을 흡수하는"

In [ ]:
answer2 = q2[['id', '대답']]
answer2.to_csv('/content/drive/MyDrive/0307-gemma/answer2.csv')

In [ ]:
test_data.head(5)

,id,질문,Answer
0,TEST_000,"방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요? 또한, 원목사이딩을 사용...","방청 페인트의 종류로는 광명단 페인트, 방청산화철 페인트, 알미늄 페인트, 역청질 ..."
1,TEST_001,도배지에 녹은 자국이 발생하는 주된 원인과 그 해결 방법은 무엇인가요?,"도배지에 녹은 자국이 발생하는 것은 녹 오염으로, 주로 습기와 온도의 변화에 기인합..."
2,TEST_002,"큐블럭의 단점을 알려주세요. 또한, 압출법 단열판을 사용하는 것의 장점은 무엇인가요?","큐블럭은 일반 벽돌에 비해 높은 비용과 균열이 생기는 염려점이 있습니다. 또한, 습..."
3,TEST_003,"철골구조를 사용하는 고층 건물에서, 단열 효과를 높이기 위한 시공 방법은 무엇이 있...","철골 구조는 뼈대 형태의 건축물로, 열 전달을 차단하여 단열 효율이 떨어집니다. 따..."
4,TEST_004,도배지의 완전한 건조를 위해 몇 주 동안 기다려야 하나요?,완전한 도배지 건조를 위해서는 일반적으로 28일 이상의 시간이 필요합니다. 그러나 ...


# Submission

In [ ]:
!pip install -q -U sentence-transformers

In [ ]:
from sentence_transformers import SentenceTransformer # SentenceTransformer Version 2.2.2

# Embedding Vector 추출에 활용할 모델(distiluse-base-multilingual-cased-v1) 불러오기
model = SentenceTransformer('distiluse-base-multilingual-cased-v1')

# 생성한 모든 응답(답변)으로부터 Embedding Vector 추출
preds = test_data["Answer"]
pred_embeddings = model.encode(preds)
pred_embeddings.shape

In [ ]:
submit = pd.read_csv('/content/drive/MyDrive/sample_submission.csv')
# 제출 양식 파일(sample_submission.csv)을 활용하여 Embedding Vector로 변환한 결과를 삽입
submit.iloc[:,1:] = pred_embeddings

In [ ]:
submit.to_csv('/content/drive/MyDrive/0307-gemma/submit-0307.csv')